In [1]:
import numpy as np
from numba import jit, vectorize, float32, float64
import pandas as pd
from smartFormat import numFmt2, sepThreeTens, lowPrec

# Test speed of tensor operations

## Create test data
This is equivalent to the first-dimension-concatenated array that results from $\nu_e$ and $\nu_\mu$ fluxes binned at $400 \;E \times 400 \cos\theta$ bins.

The resulting *input* array is then $2\times400\times400$, and the transform that must (effectively) left-multiply that is $3\times 2\times400\times400$.

This hopefully represents a realistic scenario for performing an accurate oscillation calculation.

In [2]:
np.random.seed(0)
xform = np.require(
    a=np.random.random_sample((3, 2, 400, 400)),
    dtype=np.float64,
    requirements=['C_CONTIGUOUS', 'ALIGNED']
)
inputs = np.require(
    a=np.random.random_sample((2, 400, 400)),
    dtype=np.float64,
    requirements=['C_CONTIGUOUS', 'ALIGNED']
)
xform_fp32 = np.array(xform, dtype=np.float32)
inputs_fp32 = np.array(inputs, dtype=np.float32)

In [3]:
print 'xform.dtype =', xform.dtype
print 'xform.flags =\n', xform.flags
print 'inputs.dtype =', inputs.dtype
print 'inputs.flags =\n', inputs.flags

xform.dtype = float64
xform.flags =
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False
inputs.dtype = float64
inputs.flags =
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False


In [4]:
print 'xform_fp32.dtype =', xform_fp32.dtype
print 'xform_fp32.flags =\n', xform_fp32.flags
print 'inputs_fp32.dtype =', inputs_fp32.dtype
print 'inputs_fp32.flags =\n', inputs_fp32.flags

xform_fp32.dtype = float32
xform_fp32.flags =
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False
inputs_fp32.dtype = float32
inputs_fp32.flags =
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False


## Numpy using einsum

### Float64 math on float64 inputs/transforms

In [5]:
ein_64m_64op = %timeit -r 10 -q -o np.einsum('ij..., j...', xform, inputs, dtype=np.float64, casting='unsafe');

In [6]:
ein_64m_64op_med = np.median(ein_64m_64op.all_runs) / ein_64m_64op.loops
print 'Median time, einsum FP64 math / FP64 operands:', \
        simpleFormat(ein_64m_64op_med) + ' sec'


Median time, einsum FP64 math / FP64 operands: 8.437e-3 sec


In [7]:
output_einsum = np.einsum('ij..., j...', xform, inputs)
print output_einsum.shape

(400, 400, 3)


check that it's doing what I want it to do

In [8]:
x = xform[:,:,1,10]
x

array([[ 0.01560606,  0.18573089],
       [ 0.10818345,  0.34420619],
       [ 0.74177861,  0.28394148]])

In [9]:
i = inputs[:,1,10]
i

array([ 0.89478297,  0.46917306])

In [10]:
o = np.dot(x, i)
o

array([ 0.10110397,  0.25829298,  0.79694856])

In [11]:
output_einsum[1,10,:]

array([ 0.10110397,  0.25829298,  0.79694856])

In [12]:
np.all(o == output_einsum[1,10,:])

True

### Float32 math on float64 inputs/transforms

In [13]:
ein_32m_64op = %timeit -r 10 -q -o np.einsum('ij..., j...', xform, inputs, dtype=np.float32, casting='unsafe');

In [14]:
ein_32m_64op_med = np.median(ein_32m_64op.all_runs) / ein_32m_64op.loops
print 'Median time, einsum FP32 math / FP64 operands:', \
        simpleFormat(ein_32m_64op_med) + ' sec'
print simpleFormat(ein_32m_64op_med / ein_64m_64op_med*100)+'% of ein64_64'


Median time, einsum FP32 math / FP64 operands: 5.233e-3 sec
62.02% of ein64_64


### Float32 math on float32 inputs/transforms

In [15]:
ein_32m_32op = %timeit -r 10 -q -o np.einsum('ij..., j...', xform_fp32, inputs_fp32, dtype=np.float32, casting='no');

In [16]:
ein_32m_32op_med = np.median(ein_32m_32op.all_runs) / ein_32m_32op.loops
print 'Median time, einsum FP32 math / FP32 operands:', \
        simpleFormat(ein_32m_32op_med) + ' sec'
print simpleFormat(ein_32m_32op_med / ein_64m_64op_med*100)+'% of ein64_64'

Median time, einsum FP32 math / FP32 operands: 2.812e-3 sec
33.33% of ein64_64


## Python looping

### Float64 math / float64 operands

In [17]:
def apply_python_fp64(inputs, transform):
    N_k = inputs.shape[1]
    N_l = inputs.shape[2]
    output = np.empty((N_k, N_l, 3), np.float64)
    for k in range(N_k):
        for l in range(N_l):
            output[k,l,0] = (
                transform[0,0,k,l]*inputs[0,k,l] +
                transform[0,1,k,l]*inputs[1,k,l]
            )
            output[k,l,1] = (
                transform[1,0,k,l]*inputs[0,k,l] +
                transform[1,1,k,l]*inputs[1,k,l]
            )
            output[k,l,2] = (
                transform[2,0,k,l]*inputs[0,k,l] +
                transform[2,1,k,l]*inputs[1,k,l]
            )
    return output

In [18]:
py_64m_64op = %timeit -r 5 -q -o apply_python_fp64(inputs, xform);

In [19]:
py_64m_64op_med = np.median(py_64m_64op.all_runs) / py_64m_64op.loops
print 'Median time, Python FP64 math / FP64 operands:', \
        simpleFormat(py_64m_64op_med) + ' sec'
print simpleFormat(py_64m_64op_med / ein_64m_64op_med*100)+'% of ein64_64'

Median time, Python FP64 math / FP64 operands: 7.658e-1 sec
9076% of ein64_64


In [21]:
output_python = apply_python_fp64(inputs, xform)
np.all(output_python == output_einsum)

True

### Float32 math / float32 operands

In [22]:
def apply_python_fp32(inputs, transform):
    N_k = inputs.shape[1]
    N_l = inputs.shape[2]
    output = np.empty((N_k, N_l, 3), np.float32)
    for k in range(N_k):
        for l in range(N_l):
            output[k,l,0] = (
                transform[0,0,k,l]*inputs[0,k,l] +
                transform[0,1,k,l]*inputs[1,k,l]
            )
            output[k,l,1] = (
                transform[1,0,k,l]*inputs[0,k,l] +
                transform[1,1,k,l]*inputs[1,k,l]
            )
            output[k,l,2] = (
                transform[2,0,k,l]*inputs[0,k,l] +
                transform[2,1,k,l]*inputs[1,k,l]
            )
    return output

In [23]:
py_32m_32op = %timeit -r 5 -q -o apply_python_fp32(inputs_fp32, xform_fp32);

In [24]:
py_32m_32op_med = np.median(py_32m_32op.all_runs) / py_32m_32op.loops
print 'Median time, Python FP32 math / FP32 operands:', \
        simpleFormat(py_32m_32op_med) + ' sec'
print simpleFormat(py_32m_32op_med / ein_64m_64op_med*100)+'% of ein64_64'

Median time, Python FP32 math / FP32 operands: 7.698e-1 sec
9124% of ein64_64


## Numba

### Float64 math on float64 operands

In [25]:
@jit("float64[:,:,:](float64[:,:,:], float64[:,:,:,:])", nopython=False, nogil=True, cache=True)
def apply_numba_fp64(inputs, transform):
    N_k = inputs.shape[1]
    N_l = inputs.shape[2]
    output = np.empty((N_k, N_l, 3), float64)
    for k in range(N_k):
        for l in range(N_l):
            output[k,l,0] = (
                transform[0,0,k,l]*inputs[0,k,l] +
                transform[0,1,k,l]*inputs[1,k,l]
            )
            output[k,l,1] = (
                transform[1,0,k,l]*inputs[0,k,l] +
                transform[1,1,k,l]*inputs[1,k,l]
            )
            output[k,l,2] = (
                transform[2,0,k,l]*inputs[0,k,l] +
                transform[2,1,k,l]*inputs[1,k,l]
            )
    return output

In [26]:
nu_64m_64op = %timeit -r 10 -q -o apply_numba_fp64(inputs, xform)

In [27]:
nu_64m_64op_med = np.median(nu_64m_64op.all_runs) / nu_64m_64op.loops
print 'Median time, Numba FP64 math / FP64 operands:', \
        simpleFormat(nu_64m_64op_med) + ' sec'
print simpleFormat(nu_64m_64op_med / ein_64m_64op_med*100)+'% of ein64_64'

Median time, Numba FP64 math / FP64 operands: 5.14e-3 sec
60.92% of ein64_64


In [28]:
output_numba = apply_numba_fp64(inputs, xform)
np.all(output_numba == output_einsum)

True

### Float32 math on float32 operands

In [29]:
@jit("float32[:,:,:](float32[:,:,:], float32[:,:,:,:])", nopython=True, nogil=True, cache=True)
def apply_numba_fp32(inputs, transform):
    N_k = inputs.shape[1]
    N_l = inputs.shape[2]
    output = np.empty((N_k, N_l, 3), float32)
    for k in range(N_k):
        for l in range(N_l):
            output[k,l,0] = (
                transform[0,0,k,l]*inputs[0,k,l] +
                transform[0,1,k,l]*inputs[1,k,l]
            )
            output[k,l,1] = (
                transform[1,0,k,l]*inputs[0,k,l] +
                transform[1,1,k,l]*inputs[1,k,l]
            )
            output[k,l,2] = (
                transform[2,0,k,l]*inputs[0,k,l] +
                transform[2,1,k,l]*inputs[1,k,l]
            )
    return output

In [30]:
nu_32m_32op = %timeit -r 10 -q -o apply_numba_fp32(inputs_fp32, xform_fp32)

In [31]:
nu_32m_32op_med = np.median(nu_32m_32op.all_runs) / nu_32m_32op.loops
print 'Median time, Numba FP32 math / FP32 operands:', \
        simpleFormat(nu_32m_32op_med) + ' sec'
print simpleFormat(nu_32m_32op_med / ein_64m_64op_med*100)+'% of ein64_64'

Median time, Numba FP32 math / FP32 operands: 2.396e-3 sec
28.39% of ein64_64


# What about axes ordering?

How are these affected if we change the order of the axes? I.e., keep C memory layout, but join by flavor on the *last* dimension rather than the first.

In [32]:
np.random.seed(0)
xform = np.array(np.random.random_sample((400, 400, 3, 2)),
                 dtype=np.float64)
inputs = np.array(np.random.random_sample((400, 400, 2)),
                  dtype=np.float64)

xform_fp32 = np.array(xform, dtype=np.float32)
inputs_fp32 = np.array(inputs, dtype=np.float32)

In [33]:
np.random.seed(0)
xform = np.require(
    a=np.random.random_sample((400, 400, 3, 2)),
    dtype=np.float64,
    requirements=['C_CONTIGUOUS', 'ALIGNED']
)
inputs = np.require(
    a=np.random.random_sample((400, 400, 2)),
    dtype=np.float64,
    requirements=['C_CONTIGUOUS', 'ALIGNED']
)
xform_fp32 = np.array(xform, dtype=np.float32)
inputs_fp32 = np.array(inputs, dtype=np.float32)

In [34]:
print 'xform.dtype =', xform.dtype
print 'xform.flags =\n', xform.flags
print 'inputs.dtype =', inputs.dtype
print 'inputs.flags =\n', inputs.flags

xform.dtype = float64
xform.flags =
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False
inputs.dtype = float64
inputs.flags =
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False


In [35]:
print 'xform_fp32.dtype =', xform_fp32.dtype
print 'xform_fp32.flags =\n', xform_fp32.flags
print 'inputs_fp32.dtype =', inputs_fp32.dtype
print 'inputs_fp32.flags =\n', inputs_fp32.flags

xform_fp32.dtype = float32
xform_fp32.flags =
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False
inputs_fp32.dtype = float32
inputs_fp32.flags =
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False


In [36]:
ein_64m_64op_sw = %timeit -r 10 -q -o np.einsum('...ij, ...j', xform, inputs, dtype=np.float64, casting='unsafe');

In [37]:
ein_64m_64op_sw_med = np.median(ein_64m_64op_sw.all_runs) / ein_64m_64op_sw.loops
print 'Median time, einsum FP64 math / FP64 operands swapped axes:', \
        simpleFormat(ein_64m_64op_sw_med) + ' sec'
print simpleFormat(ein_64m_64op_sw_med / ein_64m_64op_med*100)+'% of ein64_64'

Median time, einsum FP64 math / FP64 operands swapped axes: 7.844e-3 sec
92.97% of ein64_64


In [38]:
ein_32m_64op_sw = %timeit -r 10 -q -o np.einsum('...ij, ...j', xform, inputs, dtype=np.float32, casting='unsafe');

In [39]:
ein_32m_64op_sw_med = np.median(ein_32m_64op_sw.all_runs) / ein_32m_64op_sw.loops
print 'Median time, einsum FP32 math / FP64 operands swapped axes:', \
        simpleFormat(ein_32m_64op_sw_med) + ' sec'
print simpleFormat(ein_32m_64op_sw_med / ein_64m_64op_med*100)+'% of ein64_64'

Median time, einsum FP32 math / FP64 operands swapped axes: 6.082e-2 sec
720.9% of ein64_64


In [40]:
ein_32m_32op_sw = %timeit -r 10 -q -o np.einsum('...ij, ...j', xform_fp32, inputs_fp32, dtype=np.float32, casting='unsafe');

In [41]:
ein_32m_32op_sw_med = np.median(ein_32m_32op_sw.all_runs) / ein_32m_32op_sw.loops
print 'Median time, einsum FP32 math / FP32 operands swapped axes:', \
        simpleFormat(ein_32m_32op_sw_med) + ' sec'
print simpleFormat(ein_32m_32op_sw_med / ein_64m_64op_med*100)+'% of ein64_64'

Median time, einsum FP32 math / FP32 operands swapped axes: 5.684e-3 sec
67.37% of ein64_64


### Python looping

#### 64 bit math on 64 bit operands

In [42]:
def apply_python_fp64_sw(inputs, transform):
    N_k = inputs.shape[0]
    N_l = inputs.shape[1]
    output = np.empty((N_k, N_l, 3), np.float64)
    for k in range(N_k):
        for l in range(N_l):
            output[k,l,0] = (
                transform[k,l,0,0]*inputs[k,l,0] +
                transform[k,l,0,1]*inputs[k,l,1]
            )
            output[k,l,1] = (
                transform[k,l,1,0]*inputs[k,l,0] +
                transform[k,l,1,1]*inputs[k,l,1]
            )
            output[k,l,2] = (
                transform[k,l,2,0]*inputs[k,l,0] +
                transform[k,l,2,1]*inputs[k,l,1]
            )
    return output

In [44]:
py_64m_64op_sw = %timeit -r 5 -q -o apply_python_fp64_sw(inputs, xform);

In [45]:
py_64m_64op_sw_med = np.median(py_64m_64op_sw.all_runs) / py_64m_64op_sw.loops
print 'Median time, Python FP64 math / FP64 operands swapped axes:', \
        simpleFormat(py_64m_64op_sw_med) + ' sec'
print simpleFormat(py_64m_64op_sw_med / ein_64m_64op_med*100)+'% of ein64_64'

Median time, Python FP64 math / FP64 operands swapped axes: 7.534e-1 sec
8931% of ein64_64


In [46]:
def apply_python_fp32_sw(inputs, transform):
    N_k = inputs.shape[0]
    N_l = inputs.shape[1]
    output = np.empty((N_k, N_l, 3), np.float32)
    for k in range(N_k):
        for l in range(N_l):
            output[k,l,0] = (
                transform[k,l,0,0]*inputs[k,l,0] +
                transform[k,l,0,1]*inputs[k,l,1]
            )
            output[k,l,1] = (
                transform[k,l,1,0]*inputs[k,l,0] +
                transform[k,l,1,1]*inputs[k,l,1]
            )
            output[k,l,2] = (
                transform[k,l,2,0]*inputs[k,l,0] +
                transform[k,l,2,1]*inputs[k,l,1]
            )
    return output

In [47]:
py_32m_32op_sw = %timeit -r 5 -q -o apply_python_fp32_sw(inputs_fp32, xform_fp32);

In [48]:
py_32m_32op_sw_med = np.median(py_32m_32op_sw.all_runs) / py_32m_32op_sw.loops
print 'Median time, Python FP32 math / FP32 operands swapped axes:', \
        simpleFormat(py_32m_32op_sw_med) + ' sec'
print simpleFormat(py_32m_32op_sw_med / ein_64m_64op_med*100)+'% of ein64_64'

Median time, Python FP32 math / FP32 operands swapped axes: 7.531e-1 sec
8926% of ein64_64


### Numba looping

#### 64 bit math on 64 bit operands

In [49]:
@jit("float64[:,:,:](float64[:,:,:], float64[:,:,:,:])",
     nopython=True, nogil=True, cache=True)
def apply_numba_fp64_sw(inputs, transform):
    N_k = inputs.shape[0]
    N_l = inputs.shape[1]
    output = np.empty((N_k, N_l, 3), float64)
    for k in range(N_k):
        for l in range(N_l):
            output[k,l,0] = (
                transform[k,l,0,0]*inputs[k,l,0] +
                transform[k,l,0,1]*inputs[k,l,1]
            )
            output[k,l,1] = (
                transform[k,l,1,0]*inputs[k,l,0] +
                transform[k,l,1,1]*inputs[k,l,1]
            )
            output[k,l,2] = (
                transform[k,l,2,0]*inputs[k,l,0] +
                transform[k,l,2,1]*inputs[k,l,1]
            )
    return output

In [50]:
nu_64m_64op_sw = %timeit -r 10 -q -o apply_numba_fp64_sw(inputs, xform)

In [51]:
nu_64m_64op_sw_med = np.median(nu_64m_64op_sw.all_runs) / nu_64m_64op_sw.loops
print 'Median time, Numba FP64 math / FP64 operands swapped axes:', \
        simpleFormat(nu_64m_64op_sw_med) + ' sec'
print simpleFormat(nu_64m_64op_sw_med / ein_64m_64op_med*100)+'% of ein64_64'

Median time, Numba FP64 math / FP64 operands swapped axes: 4.053e-3 sec
48.04% of ein64_64


#### 32 bit math on 32 bit operands

In [52]:
@jit("float32[:,:,:](float32[:,:,:], float32[:,:,:,:])",
     nopython=True, nogil=True, cache=True)
def apply_numba_fp32_sw(inputs, transform):
    N_k = inputs.shape[0]
    N_l = inputs.shape[1]
    output = np.empty((N_k, N_l, 3), float32)
    for k in range(N_k):
        for l in range(N_l):
            output[k,l,0] = (
                transform[k,l,0,0]*inputs[k,l,0] +
                transform[k,l,0,1]*inputs[k,l,1]
            )
            output[k,l,1] = (
                transform[k,l,1,0]*inputs[k,l,0] +
                transform[k,l,1,1]*inputs[k,l,1]
            )
            output[k,l,2] = (
                transform[k,l,2,0]*inputs[k,l,0] +
                transform[k,l,2,1]*inputs[k,l,1]
            )
    return output

In [53]:
nu_32m_32op_sw = %timeit -r 10 -q -o apply_numba_fp32_sw(inputs_fp32, xform_fp32)

In [54]:
nu_32m_32op_sw_med = np.median(nu_32m_32op_sw.all_runs) / nu_32m_32op_sw.loops
print 'Median time, Numba FP32 math / FP32 operands swapped axes:', \
        simpleFormat(nu_32m_32op_sw_med) + ' sec'
print simpleFormat(nu_32m_32op_sw_med / ein_64m_64op_med*100)+'% of ein64_64'

Median time, Numba FP32 math / FP32 operands swapped axes: 1.999e-3 sec
23.7% of ein64_64


# Show summary of timing results

Tabulate the results for original axes ordering.

In [55]:
timings = [
    {'Python FP64math FP64op': py_64m_64op_med},
    {'Python FP32math FP32op': py_32m_32op_med},
    {'einsum FP64math FP64op': ein_64m_64op_med},
    {'einsum FP32math FP64op': ein_32m_64op_med},
    {'einsum FP32math FP32op': ein_32m_32op_med},
    {'Numba FP64math FP64op':  nu_64m_64op_med},
    {'Numba FP32math FP32op':  nu_32m_32op_med}
]
timings = pd.DataFrame(pd.Series(
    [t.values()[0] for t in timings],
    [t.keys()[0] for t in timings],
)).T;

Tabulate the results for swapped axes ordering.

In [57]:
timings_sw = [
    {'Python FP64math FP64op axswp': py_64m_64op_sw_med},
    {'Python FP64math FP32op axswp': py_32m_32op_sw_med},
    {'einsum FP64math FP64op axswp': ein_64m_64op_sw_med},
    {'einsum FP32math FP64op axswp': ein_32m_64op_sw_med},
    {'einsum FP32math FP32op axswp': ein_32m_32op_sw_med},
    {'Numba FP64math FP64op axswp':  nu_64m_64op_sw_med},
    {'Numba FP32math FP32op axswp':  nu_32m_32op_sw_med}
]
timings_sw = pd.DataFrame(pd.Series(
    [t.values()[0] for t in timings_sw],
    [t.keys()[0] for t in timings_sw],
)).T;

## Absolute timings (sec)

### Original axes ordering (flavor concatenated on first dimension)

In [58]:
timings

,Python FP64math FP64op,Python FP32math FP32op,einsum FP64math FP64op,einsum FP32math FP64op,einsum FP32math FP32op,Numba FP64math FP64op,Numba FP32math FP32op
0,0.765771,0.769782,0.008437,0.005233,0.002812,0.00514,0.002396


### Swapped axes ordering (flavor concatenated on last dimension)

In [59]:
timings_sw

,Python FP64math FP64op axswp,Python FP64math FP32op axswp,einsum FP64math FP64op axswp,einsum FP32math FP64op axswp,einsum FP32math FP32op axswp,Numba FP64math FP64op axswp,Numba FP32math FP32op axswp
0,0.7535,0.753098,0.007844,0.060819,0.005684,0.004053,0.001999


## Timings as fraction of einsum 64m, 64op, orig. axes

### Original axes ordering (flavor concatenated on first dimension)

In [60]:
timings / timings['einsum FP64math FP64op'].values

,Python FP64math FP64op,Python FP32math FP32op,einsum FP64math FP64op,einsum FP32math FP64op,einsum FP32math FP32op,Numba FP64math FP64op,Numba FP32math FP32op
0,90.762561,91.23798,1,0.620226,0.333314,0.609178,0.283941


### Swapped axes

In [61]:
timings_sw / timings['einsum FP64math FP64op'].values

,Python FP64math FP64op axswp,Python FP64math FP32op axswp,einsum FP64math FP64op axswp,einsum FP32math FP64op axswp,einsum FP32math FP32op axswp,Numba FP64math FP64op axswp,Numba FP32math FP32op axswp
0,89.308156,89.260512,0.929699,7.208568,0.673736,0.480363,0.236975


# Computer used for test

In [62]:
!!lscpu

['Architecture:          x86_64',
 'CPU op-mode(s):        32-bit, 64-bit',
 'Byte Order:            Little Endian',
 'CPU(s):                2',
 'On-line CPU(s) list:   0,1',
 'Thread(s) per core:    1',
 'Core(s) per socket:    2',
 'Socket(s):             1',
 'NUMA node(s):          1',
 'Vendor ID:             GenuineIntel',
 'CPU family:            6',
 'Model:                 23',
 'Model name:            Intel(R) Core(TM)2 Duo CPU     P8600  @ 2.40GHz',
 'Stepping:              10',
 'CPU MHz:               2394.000',
 'CPU max MHz:           2394.0000',
 'CPU min MHz:           798.0000',
 'BogoMIPS:              4778.16',
 'Virtualization:        VT-x',
 'L1d cache:             32K',
 'L1i cache:             32K',
 'L2 cache:              3072K',
 'NUMA node0 CPU(s):     0,1']